In [1740]:
import pandas as pd
import numpy as np
from hashlib import blake2b


df=pd.read_csv('data_prueba_tecnica.csv')
df.head(5)

,id,name,company_id,amount,status,created_at,paid_at
0,48ba4bdbfb56ceebb32f2bd0263e759be942af3d,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-03-19,NaN
1,05fc6f5ac66b6ee7e4253aa5d0c2299eb47aaaf4,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,pending_payment,2019-05-06,NaN
2,2cdce231c1fc6a2061bfa2f1d978351fe217245d,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-02-22,NaN
3,81633ba310a50b673efd469c37139576982901aa,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,102.61,paid,2019-02-27,2019-02-27
4,6ccfc4c24e788e4bca448df343698782db6b0c0b,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,184.49,paid,2019-02-05,2019-02-05


In [1741]:
df.describe()

,amount
count,1.000000e+04
mean,3.000000e+30
std,3.000000e+32
min,0.000000e+00
25%,3.121000e+01
50%,6.060500e+01
75%,1.095700e+02
max,3.000000e+34


In [1742]:
df.dtypes

id             object
name           object
company_id     object
amount        float64
status         object
created_at     object
paid_at        object
dtype: object

# Limpieza de los datos del DataFrame

## Buscamos nulos en id

In [1743]:
df[df['id'].isnull()]

,id,name,company_id,amount,status,created_at,paid_at
272,NaN,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,66.16,pending_payment,2019-03-14,NaN
9915,NaN,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,55.71,paid,2019-02-13,2019-02-13
9917,NaN,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,89.36,paid,2019-04-17,2019-04-17


### La compañia MiPasajefy, tiene 3 id en nulos

Buscaremos si los id,son unicos y cuantos id's existen

In [1744]:
#Cuenta los id's unicos
df2 = df.pivot_table(index = ['id'], aggfunc ='size')
#Cuanta cuantos id's son nulos
df_null= df[df['id'].isnull()]
print(f'''
id diferentes:

{df2} 

id nulos: {len(df_null)} 
Total de registros en el DataFrame: {len(df)}''')


id diferentes:

id
0000d8d9d1cc40ec1fddc849fccf2bf801cb8ec7    1
0008c6c98fa03633ac39c5af4418df778b9a290b    1
0014b4795f8f3b7166dabedb3030e0b55f81dd22    1
00166e87789da128e7003865becffc2ebc3e69c6    1
001e3d6bb05dccc7577d10deb5f12cc368c05bcc    1
                                           ..
ffd50b0c4224ea33369dbc0bdf1ae72e614a56ef    1
ffe2c0899fb1a4c526bf79b52311da91799df858    1
ffe66386266b4e0928bb523dbf8de74ab22bdd76    1
ffecad714275b75e2a4dc049d0017fd45cee0ba7    1
ffee16f1b4edfb13577d14b8125632115e6f9a8a    1
Length: 9997, dtype: int64 

id nulos: 3 
Total de registros en el DataFrame: 10000


Arriba, podemos observar,  que el id es unico para cada fila en nuestro DataFrame, por lo que necesitaremos asignarle un id unico a nuestro a nuestros registros nulos.



In [1745]:

#df['id'].fillna(0,inplace=True)

def UniqueID(fila):
    nombre=fila['name']
    company=fila['company_id']
    valor=fila['status']
    fecha=fila['created_at']
    id=str(nombre+company+valor+fecha)
    id_bytes=id.encode()
    h=blake2b(digest_size=20)
    h.update(id_bytes)
    return h.hexdigest()

df.loc[df['id'].isnull(),'id'] =df.loc[df['id'].isnull()].apply(UniqueID, axis=1)
df.iloc[272]

id            ef07d09f55a99d773b445b9946af24216b30d602
name                                        MiPasajefy
company_id    cbf1c8b09cd5b549416d49d220a40cbd317f952e
amount                                           66.16
status                                 pending_payment
created_at                                  2019-03-14
paid_at                                            NaN
Name: 272, dtype: object

Aplicamos la funcion UniqueID la cual hace un hash blake2b con la informacion de la fila done teniamos el id nulo, este hash es unico, por lo que no coliciona con nin id que ya tenemos en nuestro DataFrame.Esto lo podemos corroborar con la siguiente linea de codigo.

In [1746]:
#Cuenta los id's unicos
df.pivot_table(index = ['id'], aggfunc ='size')

id
0000d8d9d1cc40ec1fddc849fccf2bf801cb8ec7    1
0008c6c98fa03633ac39c5af4418df778b9a290b    1
0014b4795f8f3b7166dabedb3030e0b55f81dd22    1
00166e87789da128e7003865becffc2ebc3e69c6    1
001e3d6bb05dccc7577d10deb5f12cc368c05bcc    1
                                           ..
ffd50b0c4224ea33369dbc0bdf1ae72e614a56ef    1
ffe2c0899fb1a4c526bf79b52311da91799df858    1
ffe66386266b4e0928bb523dbf8de74ab22bdd76    1
ffecad714275b75e2a4dc049d0017fd45cee0ba7    1
ffee16f1b4edfb13577d14b8125632115e6f9a8a    1
Length: 10000, dtype: int64

## Buscamos nulos en name


In [1747]:
df[df['name'].isnull()]

,id,name,company_id,amount,status,created_at,paid_at
731,ec79a21ef969c7fc6beef080ff56baf0aeeca8b5,NaN,cbf1c8b09cd5b549416d49d220a40cbd317f952e,112.52,paid,2019-02-14,2019-02-14
2199,6a6ac16d53a02ba7948bff0a534e45404e716c5b,NaN,cbf1c8b09cd5b549416d49d220a40cbd317f952e,244.88,voided,2019-05-09,NaN
2200,4740cf2624c3b929d9944cdfcb5c87e71a82ddc5,NaN,cbf1c8b09cd5b549416d49d220a40cbd317f952e,118.78,paid,2019-03-05,2019-03-06


Tenemos nulos en el la columna "name", por lo que buscaremos cuantos nombres existen y que conexion tiene con las demas columnas en el DataFrame

In [1748]:
df.pivot_table(index = ['name','company_id'], aggfunc ='size')

name            company_id                              
MiP0xFFFF       cbf1c8b09cd5b549416d49d220a40cbd317f952e       1
MiPas0xFFFF     cbf1c8b09cd5b549416d49d220a40cbd317f952e       1
MiPasajefy      *******                                        1
                cbf1c8b09cd5b549416d49d220a40cbd317f952e    9894
Muebles chidos  8f642dc67fccf861548dfe1c761ce22f795e91f0      96
dtype: int64

In [1749]:
df.pivot_table(index = ['company_id','name'], aggfunc ='size')

company_id                                name          
*******                                   MiPasajefy           1
8f642dc67fccf861548dfe1c761ce22f795e91f0  Muebles chidos      96
cbf1c8b09cd5b549416d49d220a40cbd317f952e  MiP0xFFFF            1
                                          MiPas0xFFFF          1
                                          MiPasajefy        9894
dtype: int64

De acuerdo a las 2 tablas anterirores, podemos observar la relacion que hay entre "compañia_id" y el nombre de la empresa,
Podemos observar que el nombre de la empresa esta corrompido, ya que la columna "company_id" esta asociado a 3 nombres distintos, de los cuales 2 son el comienzo del verdadero nombre.

Buscamos las coincidencias ente el company_id y enombre diferente a "MiPasajefy".

In [1750]:
df[(df['name']!='MiPasajefy')&(df['company_id']=='cbf1c8b09cd5b549416d49d220a40cbd317f952e')]

,id,name,company_id,amount,status,created_at,paid_at
731,ec79a21ef969c7fc6beef080ff56baf0aeeca8b5,NaN,cbf1c8b09cd5b549416d49d220a40cbd317f952e,112.52,paid,2019-02-14,2019-02-14
1320,60366f3cfb5245ec62143984eef4ba93cfbe3c38,MiPas0xFFFF,cbf1c8b09cd5b549416d49d220a40cbd317f952e,134.99,paid,2019-01-25,2019-01-25
1479,891732cfc6980d71a24ccba4138a3af2f14413fb,MiP0xFFFF,cbf1c8b09cd5b549416d49d220a40cbd317f952e,47.64,voided,2019-03-17,NaN
2199,6a6ac16d53a02ba7948bff0a534e45404e716c5b,NaN,cbf1c8b09cd5b549416d49d220a40cbd317f952e,244.88,voided,2019-05-09,NaN
2200,4740cf2624c3b929d9944cdfcb5c87e71a82ddc5,NaN,cbf1c8b09cd5b549416d49d220a40cbd317f952e,118.78,paid,2019-03-05,2019-03-06


De acuerdo a la informacion anteriror , podemos observar que el company_id "cbf1c8b09cd5b549416d49d220a40cbd317f952e" esta asignado a nombres incorrectos de la compañia "MiPasajefy", asi que cambimos los nombres incorrectos, por el nombre correcto de la empresa.

In [1751]:
df.loc[(df['name']!='MiPasajefy')&(df['company_id']=='cbf1c8b09cd5b549416d49d220a40cbd317f952e'),'name']='MiPasajefy'

In [1752]:
df.pivot_table(index = ['company_id','name'], aggfunc ='size')

company_id                                name          
*******                                   MiPasajefy           1
8f642dc67fccf861548dfe1c761ce22f795e91f0  Muebles chidos      96
cbf1c8b09cd5b549416d49d220a40cbd317f952e  MiPasajefy        9899
dtype: int64

Ahora podemos ver que el company_id "cbf1c8b09cd5b549416d49d220a40cbd317f952e", le pertenece solo al una empresa.
Pero ahora tenemos que la empresa "MiPasajefy" tiene asignada un company_id diferente, asi que lo cambiamos al company_id correcto.

In [1753]:
df.loc[(df['company_id']!='cbf1c8b09cd5b549416d49d220a40cbd317f952e')&(df['name']=='MiPasajefy'),'company_id']='cbf1c8b09cd5b549416d49d220a40cbd317f952e'
df.pivot_table(index = ['company_id','name'], aggfunc ='size')

company_id                                name          
8f642dc67fccf861548dfe1c761ce22f795e91f0  Muebles chidos      96
cbf1c8b09cd5b549416d49d220a40cbd317f952e  MiPasajefy        9904
dtype: int64

Ahora ya tenemos relacionadas las compnay-id con las empresas correspondientes

In [1754]:
df.groupby(['company_id','name']).apply(lambda grp: grp.count())

,,id,name,company_id,amount,status,created_at,paid_at
company_id,name,,,,,,,
8f642dc67fccf861548dfe1c761ce22f795e91f0,Muebles chidos,96,96,96,96,96,96,32
cbf1c8b09cd5b549416d49d220a40cbd317f952e,MiPasajefy,9904,9904,9904,9904,9904,9904,5977


## Buscamos si hay alguna anomalia en amount

In [1755]:
df[df['amount'].isnull()]

,id,name,company_id,amount,status,created_at,paid_at


In [1756]:
df[df['amount']>100000000000000]

,id,name,company_id,amount,status,created_at,paid_at
1515,37fd630d05e0b4c9c179eea6f88541e27adfbd07,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,2.131231e+18,pending_payment,2019-03-27,NaN
1609,a98245b3f1e8484f2e65f245ef35051822d0f4fb,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,2.132132e+16,pending_payment,2019-05-09,NaN
1896,a07c65559e91a7b08a25f6c7e9f130cdfb727a42,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.000000e+34,voided,2019-04-17,NaN


Podemos observar que la columna amount, contiene numeros que exceden nuestra restriccion de amount decimal(16,2) NOT NULL, asi que debemos transformarlos para que cumplan con la restriccion.

In [1757]:
# Limitaremos los numeros con las restricciones que ya tenmos, despues lo redondearemos a 2 decimales
df['amount'] = np.clip(df['amount'], -9999999999999, 99999999999999)
df['amount'] = np.round(df['amount'], 2)


In [1758]:
df[df['amount']>100000000000000]

,id,name,company_id,amount,status,created_at,paid_at


## Buscamos si hay algun nulo en status 

In [1759]:
df[df['status'].isnull()]

,id,name,company_id,amount,status,created_at,paid_at


No tenemos ningun nulo, pero si tenemos tipos extraños de status

In [1760]:
df.pivot_table(index = ['status'], aggfunc ='size')

status
0xFFFF                   1
charged_back            21
expired                  8
p&0x3fid                 1
paid                  5892
partially_refunded       1
pending_payment       1881
pre_authorized          18
refunded                93
voided                2084
dtype: int64

In [1761]:
df[df['status']=='paid']

,id,name,company_id,amount,status,created_at,paid_at
3,81633ba310a50b673efd469c37139576982901aa,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,102.61,paid,2019-02-27,2019-02-27
4,6ccfc4c24e788e4bca448df343698782db6b0c0b,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,184.49,paid,2019-02-05,2019-02-05
9,cdf1a7a6f5c9b0db543371f9e4bd127b451c09c9,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,115.84,paid,2019-01-22,2019-01-22
11,90820e39fa6aa60555edb6855a6c83e59a9ba271,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,81.33,paid,2019-01-12,2019-01-12
12,315fb708b31d2b98da63db63481550cf01087c75,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,230.27,paid,2019-03-29,2019-03-29
...,...,...,...,...,...,...,...
9995,4907c36c037bfa4874047e14f722329016dc3908,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,59.88,paid,2019-03-14,2019-03-14
9996,ef9ec85887418d2ccf220686312cfc4c615665d5,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,23.05,paid,2019-02-20,2019-02-20
9997,50bb3950ecde94ca3e07853003a11a413508da36,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,20.50,paid,2019-03-19,2019-03-19
9998,1b2c4e503a4a952064b70369cf8e72937c720474,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,81.42,paid,2019-03-10,2019-03-10


In [1762]:
df[df['status']=='p&0x3fid']

,id,name,company_id,amount,status,created_at,paid_at
1308,9c99ef0dc472daaf059be45c8dd24a3781fd1af7,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,82.36,p&0x3fid,2019-03-14,2019-03-14


In [1763]:
df[df['status']=='0xFFFF']

,id,name,company_id,amount,status,created_at,paid_at
3510,8ee1ad3b1736f801145e18ecf54e346c006e43d5,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,146.74,0xFFFF,2019-02-03,2019-02-03


Como podemos observar, ambas filas que tienen el nombdre del estatus alterado, tienen la caracteristica del estatus "paid", por ende , modificamos su valor para que sea "paid"

In [1764]:
df['status'] = df['status'].replace(['0xFFFF'], 'paid')
df['status'] = df['status'].replace(['p&0x3fid'], 'paid')


Ahora, ya tenemos los estatus correctos

In [1765]:
df.pivot_table(index = ['status'], aggfunc ='size')

status
charged_back            21
expired                  8
paid                  5894
partially_refunded       1
pending_payment       1881
pre_authorized          18
refunded                93
voided                2084
dtype: int64

## Buscamos si existe alguna anomalia en created_at

In [1766]:
df[df['created_at'].isnull()]

,id,name,company_id,amount,status,created_at,paid_at


In [1767]:
df.pivot_table(index = ['paid_at'], aggfunc ='size')

paid_at
2019-01-01    17
2019-01-02    45
2019-01-03    38
2019-01-04    53
2019-01-05    37
              ..
2019-05-16    49
2019-05-17    54
2019-05-18    54
2019-05-19    26
2019-05-20     3
Length: 140, dtype: int64

In [1768]:

df['created_at'] = pd.to_datetime(df['created_at'])
df['created_at'] = df['created_at'].dt.date

df.dtypes

id             object
name           object
company_id     object
amount        float64
status         object
created_at     object
paid_at        object
dtype: object

## Buscamos si existe alguna anomalia en paid_at

In [1769]:
df[df['paid_at'].isnull()]

,id,name,company_id,amount,status,created_at,paid_at
0,48ba4bdbfb56ceebb32f2bd0263e759be942af3d,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-03-19,NaN
1,05fc6f5ac66b6ee7e4253aa5d0c2299eb47aaaf4,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,pending_payment,2019-05-06,NaN
2,2cdce231c1fc6a2061bfa2f1d978351fe217245d,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-02-22,NaN
5,b25f2ff15c24ea881e676be772f4dd99891ad188,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,85.27,pending_payment,2019-01-04,NaN
6,4f9db76960dcdf6b7e2d7d71b11519e49d7b1179,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,41.00,pre_authorized,2019-05-04,NaN
...,...,...,...,...,...,...,...
9983,b7243030b5ecb367fed8f42b5961e6cf39532a63,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,30.00,pending_payment,2019-03-01,NaN
9986,6567f52b0e2c912234984fabdef6747b0d8c20c5,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-04-12,NaN
9987,18917f0b8f9d3ada2ed5d0f583d635211012d9b5,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-01-08,NaN
9990,e769564fa450e60a62d358e4d72804d361ee017f,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-05-12,NaN


Podemos observar que tenmos muchos registros que estan en nulo, por loq eu pondremos una muy antigua, casi riducla, de esta forma sabremos que aun no han pagado,la fecha por convenio sera 1999/12/12

In [1770]:
df['paid_at'].fillna('1999-12-12',inplace=True)
df[df['paid_at']=='12-12-12']

,id,name,company_id,amount,status,created_at,paid_at


Ahora ya tenmos una fecha que sabemos que significa lo mismo que nulo.

Verificamos si hay solo fechas en la columna

In [1771]:

df['paid_at'] = pd.to_datetime(df['paid_at'])
df['paid_at'] = df['paid_at'].dt.date

df.dtypes

id             object
name           object
company_id     object
amount        float64
status         object
created_at     object
paid_at        object
dtype: object

# Transformar los id y company_id en un varchar(24 NOT NULL)

Para transformar los id, usaremos blake2b, para hacer un hash unico para cada id y un hash unico para cada company_id, ambos con un largo de 24 caracteres.

In [1772]:
def IDHASH(id):
    id_bytes=id.encode()
    h=blake2b(digest_size=12)
    h.update(id_bytes)
    #print(len(h.hexdigest()))
    return h.hexdigest()

df['db_id']=df['id'].apply(IDHASH)
df['db_company_id']=df['company_id'].apply(IDHASH)

Ahora debemos tener el numero de db_company_id y names

In [1773]:
df.pivot_table(index = ['name','db_company_id'], aggfunc ='size')

name            db_company_id           
MiPasajefy      214e15c60cf3d66b79562923    9904
Muebles chidos  f52549e093bca421285219df      96
dtype: int64

Asi mismo, tenemos que tener la misma cantidad de id y db_id

In [1774]:
df.pivot_table(index = ['id','db_id'], aggfunc ='size')

id                                        db_id                   
0000d8d9d1cc40ec1fddc849fccf2bf801cb8ec7  c4a594051ee46c8bd0984620    1
0008c6c98fa03633ac39c5af4418df778b9a290b  696b4adf16368c3d70cc20e5    1
0014b4795f8f3b7166dabedb3030e0b55f81dd22  4dc97d8089abc96fe82da583    1
00166e87789da128e7003865becffc2ebc3e69c6  515a02ad2f61181c1345e646    1
001e3d6bb05dccc7577d10deb5f12cc368c05bcc  2d5f53455223be590ac0370b    1
                                                                     ..
ffd50b0c4224ea33369dbc0bdf1ae72e614a56ef  3d7234e1dd222da77a5fd17a    1
ffe2c0899fb1a4c526bf79b52311da91799df858  29879443747be3396372d656    1
ffe66386266b4e0928bb523dbf8de74ab22bdd76  a5ad446dd8ed7ebd72b66313    1
ffecad714275b75e2a4dc049d0017fd45cee0ba7  26edea853fc9903cef874125    1
ffee16f1b4edfb13577d14b8125632115e6f9a8a  2646ac810fb3eaaedeabd1d0    1
Length: 10000, dtype: int64

In [1775]:
df.pivot_table(index = ['db_id','id'], aggfunc ='size')

db_id                     id                                      
00032df5d1a72d1944eaab63  78af7f64a2264ecf8b349607f53b2c239b2b5fe5    1
000ca98e5d552931082662d2  2039e9e45ce5464af579ece287cae621876d48ab    1
00110a871a564139bef2fc8c  fc7e4075c01baa1a73315050e9d5105784395459    1
00180976e2ddc844bf282c94  6f52e27befdd6f1f6bab7784112d8feb1f88fed7    1
001de12a38a586c7ab76d939  1195272a444354432c09d4d4351b92e34da47b4d    1
                                                                     ..
ffd5c2bc300d8eea5b0d3ba3  0509a5aa5abd54cc211b12e9964a9568c8554283    1
ffe35637783de64c64174beb  3f466427e4069f3e615ed0ec475ae7a41a96cf00    1
ffe3a45a2344fe4856508aa5  0083f3c78dabddf1d8efbed0e80731908cf6bb09    1
ffeb4a277a73e51207664a24  5cdccb51221d2efa3c9b29850a871c39bc074fbb    1
fffdba1d60ad051da33e2e2b  720452cb219057205d516bbc2d7f2c018cb23830    1
Length: 10000, dtype: int64

## Creamos un nuevo CSV con la informacion que necesitamos

In [1776]:
df.to_csv("./NewData.csv")

In [1777]:
df[['db_id','db_company_id','name','amount','status','created_at','paid_at']]

,db_id,db_company_id,name,amount,status,created_at,paid_at
0,5db5526f138308320aaac8c0,214e15c60cf3d66b79562923,MiPasajefy,3.00,voided,2019-03-19,1999-12-12
1,8186c5bfd7e422740f141bcf,214e15c60cf3d66b79562923,MiPasajefy,3.00,pending_payment,2019-05-06,1999-12-12
2,0db3e82aeeb18ad6f2e9c638,214e15c60cf3d66b79562923,MiPasajefy,3.00,voided,2019-02-22,1999-12-12
3,f99b8758e9985a4bd0ed8f71,214e15c60cf3d66b79562923,MiPasajefy,102.61,paid,2019-02-27,2019-02-27
4,7d32e2189ddc4b0111b9313b,214e15c60cf3d66b79562923,MiPasajefy,184.49,paid,2019-02-05,2019-02-05
...,...,...,...,...,...,...,...
9995,efed81381f2b12974b6be1f9,214e15c60cf3d66b79562923,MiPasajefy,59.88,paid,2019-03-14,2019-03-14
9996,a095b02dc3c7d455f9ba23f7,214e15c60cf3d66b79562923,MiPasajefy,23.05,paid,2019-02-20,2019-02-20
9997,f093f83de6aac881ea0f060b,214e15c60cf3d66b79562923,MiPasajefy,20.50,paid,2019-03-19,2019-03-19
9998,c189dc594fe0e114954f651f,214e15c60cf3d66b79562923,MiPasajefy,81.42,paid,2019-03-10,2019-03-10
